In [53]:
import glob
import re
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings("ignore")

# Merge Data

In [54]:
main_data = glob.glob("data/*.csv")

In [55]:
df_list = [pd.read_csv(file) for file in main_data]
merged_df = pd.concat(df_list, ignore_index=True)

In [56]:
merged_df.to_csv("merged_data.csv", index=False)

In [57]:
df = merged_df.copy()

# Cleaning Data


In [58]:
print("Jumlah baris sebelum cleaning:", len(df))

Jumlah baris sebelum cleaning: 2731


In [59]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2731 entries, 0 to 2730
Data columns (total 26 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   id                   2728 non-null   object
 1   Video Description    2731 non-null   object
 2   Video Create Time    2731 non-null   object
 3   Video Category Type  2731 non-null   object
 4   Video Duration       2731 non-null   object
 5   Video Width          2731 non-null   object
 6   Video Height         2731 non-null   object
 7   Video URL            2731 non-null   object
 8   Video Cover          2731 non-null   object
 9   Video View Count     2731 non-null   object
 10  Video Like Count     2731 non-null   object
 11  Video Comment Count  2731 non-null   object
 12  Video Share Count    2731 non-null   object
 13  Author ID            2731 non-null   object
 14  Author Nickname      2731 non-null   object
 15  Author Username      2731 non-null   object
 16  Author

In [60]:
df["Video Create Time"] = pd.to_datetime(df["Video Create Time"], unit="s", errors="coerce")

In [61]:
columns_to_drop = [
    "Video Width", "Video Height", "Video URL", "Video Cover",
    "Author ID", "Author Avatar", "Music ID", "Music Cover", "Music Duration","Music URL"
]

df.drop(columns=columns_to_drop, inplace=True)

In [62]:
df['id'].duplicated().any()

True

In [63]:
df["id"].duplicated().sum()


111

In [64]:
df.drop_duplicates(subset="id", keep="first", inplace=True)

In [65]:
df['id'].duplicated().any()

False

In [66]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2620 entries, 0 to 2730
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   id                   2619 non-null   object        
 1   Video Description    2620 non-null   object        
 2   Video Create Time    2617 non-null   datetime64[ns]
 3   Video Category Type  2620 non-null   object        
 4   Video Duration       2620 non-null   object        
 5   Video View Count     2620 non-null   object        
 6   Video Like Count     2620 non-null   object        
 7   Video Comment Count  2620 non-null   object        
 8   Video Share Count    2620 non-null   object        
 9   Author Nickname      2620 non-null   object        
 10  Author Username      2620 non-null   object        
 11  Author Bio           2620 non-null   object        
 12  Author Verified      2620 non-null   object        
 13  Music Title          2620 non-null   o

In [67]:
numeric_cols = [
    "Video View Count", "Video Like Count",
    "Video Comment Count", "Video Share Count",
    "Video Duration"
]

for col in numeric_cols:
    df[col] = pd.to_numeric(df[col], errors="coerce")


In [68]:
def bersihkan_karakter_aneh(text):
    if pd.isna(text):
        return text
    return re.sub(r'[^\x00-\x7F]+', '', str(text))

for col in df.select_dtypes(include='object').columns:
    df[col] = df[col].apply(bersihkan_karakter_aneh)


In [69]:
df = df[df["Author Verified"] != "Not Available"]

In [70]:
df.isna().sum().sort_values(ascending=False)


id                     0
Video Description      0
Video Create Time      0
Video Category Type    0
Video Duration         0
Video View Count       0
Video Like Count       0
Video Comment Count    0
Video Share Count      0
Author Nickname        0
Author Username        0
Author Bio             0
Author Verified        0
Music Title            0
Music Author           0
Video Page URL         0
dtype: int64

In [71]:
for col in df.select_dtypes(include='object').columns:
    df.loc[:, col] = df[col].str.strip()

In [72]:
# 2. Ganti string kosong jadi NaN (biar fillna bisa kerja)
df.replace(r'^\s*$', np.nan, regex=True, inplace=True)

# 3. Isi nilai kosong (NaN)
df["Video Description"].fillna("missing_description", inplace=True)
df["Author Nickname"].fillna(df["Author Username"], inplace=True)
df["Author Bio"].fillna("No bio yet", inplace=True)
df["Music Title"].fillna("unknown", inplace=True)
df["Music Author"].fillna("unknown", inplace=True)

# 4. Ganti string literal "Not Available" ke placeholder
df.loc[:, "Video Description"] = df["Video Description"].replace("Not Available", "missing_description")
df.loc[:, "Video Category Type"] = df["Video Category Type"].replace("Not Available", "unknown")

In [73]:
exclude_cols = ["Video Page URL"]

for col in df.select_dtypes(include='object').columns:
    if col not in exclude_cols:
        df.loc[:, col] = df[col].str.strip().str.lower()


In [74]:
df["Music Author"] = df["Music Author"].replace("not available", "unknown")
df["Music Title"] = df["Music Title"].replace("not available", "unknown")

In [75]:
df["Upload Hour"] = df["Video Create Time"].dt.hour
df["Upload Day"] = df["Video Create Time"].dt.day_name()


In [76]:
df["Author Verified"] = df["Author Verified"].astype("category")
df["Video Category Type"] = df["Video Category Type"].astype("category")
df["Upload Day"] = df["Upload Day"].astype("category")


In [77]:
df.drop(columns=["id"], inplace=True)


In [78]:
df = df[[
    "Video Page URL", "Video Create Time", "Video Category Type",
    "Upload Hour", "Upload Day", "Video Duration",
    "Video View Count", "Video Like Count", "Video Comment Count", "Video Share Count",
    "Author Username", "Author Nickname", "Author Bio", "Author Verified",
    "Video Description", "Music Title", "Music Author"
]]


In [79]:
df["Author Verified"] = df["Author Verified"].map({"yes": True, "no": False})


In [80]:
print("Jumlah baris setelah cleaning:", len(df))

df.to_csv("cleaned_data.csv", index=False)


Jumlah baris setelah cleaning: 2600
